# Data set description
## Content

This dataset contains scheduled and actual departure and arrival times reported by certified U.S. air carriers that account for at least one percent of domestic scheduled passenger revenues. The data is collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS).
This dataset covers all the flights from January 2019 to September 2020 for all U.S region, which is the latest date that has available data.
For clarification, according to the U.S. Department of Transportation (DOT), a flight is considered as delayed if it arrives at the destination gate 15 minutes or more after its scheduled arrival time.
## Dataset attributes
- FlightDate: Date of flight
- IATA_CODE_Reporting_Airline: Code assigned by IATA and commonly used to identify a carrier.
- Flight_Number_Reporting_Airline: Flight Number.
- Origin: Origin airport
- Dest: Destination airport
- CRSDepTime: CRS Departure Time (local time: hhmm)
- DepTime: Actual Departure Time (local time: hhmm)
- DepDelay: Difference in minutes between scheduled and actual departure time. Early departures show negative numbers.
- TaxiOut: Taxi Out Time, in Minutes.
- WheelsOff: Wheels Off Time (local time: hhmm)
- WheelsOn: Wheels On Time (local time: hhmm)
- TaxiIn: Taxi In Time, in Minutes.
- CRSArrTime: CRS Arrival Time (local time: hhmm)
- ArrTime: Actual Arrival Time (local time: hhmm)
- ArrDelay: Difference in minutes between scheduled and actual arrival time. Early arrivals show negative numbers.
- Cancelled: Cancelled Flight Indicator (1=Yes)
- CancellationCode: Specifies The Reason For Cancellation.
- Diverted: Diverted Flight Indicator (1=Yes)
- CRSElapsedTime: CRS Elapsed Time of Flight, in Minutes
- ActualElapsedTime: Elapsed Time of Flight, in Minutes
- AirTime: Flight Time, in Minutes.
- Distance: Distance between airports (miles)
- CarrierDelay: Carrier Delay, in Minutes.
- WeatherDelay: Weather Delay, in Minutes.
- NASDelay: National Air System Delay, in Minutes.
- SecurityDelay: Security Delay, in Minutes.
- LateAircraftDelay: Late Aircraft Delay, in Minutes

## Attribute explanation

- CRSCRSDepTime/CRSArrTime are the computerized reservation system departure/arrival time. This is the scheduled time for departure/arrival of the flight.
- WheelsOff/WheelsOn: is the time when the wheels of the aircraft leave the ground at the origin airport, and is the time when the wheels of the aircraft touch the ground at the destination airport.
- TaxiIn: The time elapsed between wheels down and arrival at the destination airport gate.
- TaxiOut: The time elapsed between departure from the origin airport gate and wheels off.
- CRSElapsedTime/ActualElapsedTime: The scheduled/actual time computed from gate departure time to gate arrival time.
- CarrierDelay/WeatherDelay/NASDelay/SecurityDelay/LateAircraftDelay: delay factors, in minutes.
- CancellationCode: reason for cancellation (A = carrier, B = weather, C = NAS, D = security)

For more information, we can take a look at this figure:

![Image of aircraft travel](https://live.staticflickr.com/65535/50720049521_083fa9863e.jpg)

# Loading dataset & dataset info

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import glob
import os
from dask import compute
import plotly.graph_objects as go
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

init_notebook_mode(connected=True)
cf.go_offline(connected=True)
cf.set_config_file(theme="pearl")
pd.set_option('display.max_columns', 30)
%matplotlib inline

In [2]:
from dask.distributed import Client
client=Client(n_workers=8, threads_per_worker=4, memory_limit="8GB")

In [3]:
client

Client Scheduler: tcp://127.0.0.1:58872 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 64.00 GB


In [4]:
data = dd.read_csv(glob.glob(os.path.join("*.csv")), parse_dates=["FL_DATE"],blocksize=50000000)
data.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2019-04-01,NK,781,BWI,MCO,1020,1037.0,17.0,11.0,1048.0,1249.0,16.0,1245,1305.0,20.0,0.0,NaN,0.0,145.0,148.0,121.0,787.0,0.0,0.0,3.0,0.0,17.0,NaN
1,2019-04-01,NK,781,DTW,BWI,755,830.0,35.0,12.0,842.0,942.0,6.0,928,948.0,20.0,0.0,NaN,0.0,93.0,78.0,60.0,409.0,20.0,0.0,0.0,0.0,0.0,NaN
2,2019-04-01,NK,782,BWI,DTW,1931,1946.0,15.0,8.0,1954.0,2105.0,29.0,2108,2134.0,26.0,0.0,NaN,0.0,97.0,108.0,71.0,409.0,10.0,0.0,11.0,0.0,5.0,NaN
3,2019-04-01,NK,782,MCO,BWI,1623,1651.0,28.0,11.0,1702.0,1841.0,5.0,1841,1846.0,5.0,0.0,NaN,0.0,138.0,115.0,99.0,787.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-04-01,NK,788,LAS,DTW,2320,2348.0,28.0,23.0,11.0,632.0,11.0,622,643.0,21.0,0.0,NaN,0.0,242.0,235.0,201.0,1749.0,21.0,0.0,0.0,0.0,0.0,NaN


In [5]:
n_row = data.shape[0]
n_col = data.shape[1]
print(f'There are total {n_row.compute()} rows and {n_col} columns in the dataset')

There are total 11022561 rows and 28 columns in the dataset


# Dataset cleaning
When importing the dataset, we realized that there's one extra column in the dataset, it is the last one and contains only missing value. Therefore, this column will be dropped first.

In [6]:
data = data.drop("Unnamed: 27", axis=1)

## Drop duplicated row and unnecessary column

The OP_CARRIER_FL_NUM is the code assigned by IATA and commonly used to identify a carrier. As the same code may have been assigned to different carriers over time, the code is not always unique. Therefore, we will remove this column.

In [7]:
data = data.drop("OP_CARRIER_FL_NUM", axis=1)

In [8]:
data.drop_duplicates(inplace=True)
n_row = data.shape[0]
n_col = data.shape[1]
print(f'There are total {n_row.compute()} rows and {n_col} columns in the dataset after removing duplicates')

There are total 11022561 rows and 26 columns in the dataset after removing duplicates


We can see that there is no duplicate in the dataset.
## Handle missing values
In this dataset, we have three cases of missing values:
- Flight depart on time or earlier will leads to missing value in columns with delay factors.
- Flight is canceled/diverted, which leads to missing values in other columns.
- And finally, the flight with missing information due other reason than above ones. 

In [9]:
data.isnull().sum().compute()

FL_DATE                       0
OP_CARRIER                    0
ORIGIN                        0
DEST                          0
CRS_DEP_TIME                  0
DEP_TIME                 402402
DEP_DELAY                402487
TAXI_OUT                 406871
WHEELS_OFF               406871
WHEELS_ON                411527
TAXI_IN                  411527
CRS_ARR_TIME                  0
ARR_TIME                 411526
ARR_DELAY                433202
CANCELLED                     0
CANCELLATION_CODE      10614343
DIVERTED                      0
CRS_ELAPSED_TIME            139
ACTUAL_ELAPSED_TIME      433202
AIR_TIME                 433202
DISTANCE                      0
CARRIER_DELAY           9301831
WEATHER_DELAY           9301831
NAS_DELAY               9301831
SECURITY_DELAY          9301831
LATE_AIRCRAFT_DELAY     9301831
dtype: int64

In [10]:
def check_null(col):
    num_canceled = data[data[col].isnull()]["CANCELLED"].sum()
    num_diverted = data[data[col].isnull()]["DIVERTED"].sum()
    total_null = data[data[col].isnull()]["FL_DATE"].count()
    
    return (num_canceled+num_diverted-total_null).compute()

In [11]:
null_col = ["DEP_TIME","DEP_DELAY", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "TAXI_IN", "CRS_ARR_TIME", "ARR_TIME", 
            "ARR_DELAY", "CRS_ELAPSED_TIME", "AIR_TIME", "ACTUAL_ELAPSED_TIME"]
a = [check_null(col) for col in null_col]
a

[0.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, -1.0]

In [12]:
[data[(data[null_col[i]].isnull())&(data["CANCELLED"]==0)&(data["DIVERTED"]==0)].compute().index if a[i] !=0 else 0 for i in range(len(a))]

[0,
 0,
 0,
 0,
 Int64Index([175311], dtype='int64'),
 Int64Index([175311], dtype='int64'),
 0,
 Int64Index([175311], dtype='int64'),
 Int64Index([175311], dtype='int64'),
 0,
 Int64Index([175311], dtype='int64'),
 Int64Index([175311], dtype='int64')]

There are a lot of missing values among columns, however, almost all of them are due to being canceled or diverted except one flight that had not been canceled or diverted. There is still one row that has missing value but has no status of being canceled or diverted. Therefore, this column will be dropped from the dataset.

In [13]:
data = data[~((data["WHEELS_ON"].isnull())&(data["TAXI_IN"].isnull())&
             (data["ARR_TIME"].isnull())&(data["ARR_DELAY"].isnull())&
            (data["AIR_TIME"].isnull())&(data["ACTUAL_ELAPSED_TIME"].isnull())&
            (data["CANCELLED"]==0)&(data["DIVERTED"]==0))]

We further check for rows that has arrival delay greater than 15 minutes but do not have information on the time for delay factors.

In [14]:
b = data[["ARR_DELAY", "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]]
b[(b["ARR_DELAY"]>15)&(b[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].sum(axis=1)==0)].compute()

,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
14989,509.0,NaN,NaN,NaN,NaN,NaN


So, there is another row that should be dropped from the dataset.

In [15]:
data = data[~((b["ARR_DELAY"]>15)&
              (b[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].sum(axis=1)==0))]

After dropping all the unreasonable rows, we can fill remain missing values with 0 for delay factors.

In [16]:
data[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]] = data[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].fillna(0)

## Reformatting and Re-ordering

In [17]:
data.dtypes

FL_DATE                datetime64[ns]
OP_CARRIER                     object
ORIGIN                         object
DEST                           object
CRS_DEP_TIME                    int64
DEP_TIME                      float64
DEP_DELAY                     float64
TAXI_OUT                      float64
WHEELS_OFF                    float64
WHEELS_ON                     float64
TAXI_IN                       float64
CRS_ARR_TIME                    int64
ARR_TIME                      float64
ARR_DELAY                     float64
CANCELLED                     float64
CANCELLATION_CODE              object
DIVERTED                      float64
CRS_ELAPSED_TIME              float64
ACTUAL_ELAPSED_TIME           float64
AIR_TIME                      float64
DISTANCE                      float64
CARRIER_DELAY                 float64
WEATHER_DELAY                 float64
NAS_DELAY                     float64
SECURITY_DELAY                float64
LATE_AIRCRAFT_DELAY           float64
dtype: objec

There are several columns indicate the time in format of hhmm, such as:
- CRS_DEP_TIME, DEP_TIME
- CRS_ARR_TIME, ARR_TIME
- WHEELS_OFF, WHEELS_ON

For these columns, we will convert time to datetime format. Because the converting processes will be the same for some other columns, it is better to define a function for changing the format instead of re-coding over and over again.

In [18]:
def int_to_timedelta(col):
    hours = col//100
    minutes = col%100
    hours_timedelta = pd.to_timedelta(hours,unit="h")
    minute_timedelta = pd.to_timedelta(minutes,unit="m")
    
    return (hours_timedelta + minute_timedelta)

We will also define another function to convert minute to timedelta object to use with columns with minutes format.

In [19]:
def min_to_timedelta(col):
    return pd.to_timedelta(col,unit="m")

Since there are difference between timezones, so we will create a timezone column for later calculation.

In [20]:
timezone = data["WHEELS_ON"].map_partitions(int_to_timedelta) - data["WHEELS_OFF"].map_partitions(int_to_timedelta) - \
data["AIR_TIME"].map_partitions(min_to_timedelta)

For the CRS_DEP_TIME column, we will combine this information with flight date and drop the column since it is scheduled time and nothing would change for this information.

In [21]:
data["FL_DATE"] = data["FL_DATE"] + data["CRS_DEP_TIME"].map_partitions(int_to_timedelta)
data = data.drop("CRS_DEP_TIME", axis=1)

For the DEP_TIME column, because there are flights that are scheduled at midnight but only depart next day due to the delay. Therefore, the date of taking off would be different from the scheduled date. We have the information of delay time between CRS departure and actual time so we can just simply convert the departure delay time to timedelta object and add it to CRS departure time. However, the data of departure delay time are in suitable type for analyzing (minutes), therefore, we don't need replace this data with timedelta format.

In [22]:
data["DEP_TIME"] = data["FL_DATE"] + data["DEP_DELAY"].map_partitions(min_to_timedelta)

The same processes are applied for WHEELS_OFF but for this column, the date will be the same as DEP_TIME instead of FL_DATE and the timedelta will be TAXI_OUT.

In [23]:
data["WHEELS_OFF"] = data["DEP_TIME"] + data["TAXI_OUT"].map_partitions(min_to_timedelta)

For the arrival related columns, the difference in timezone affects the actual arrival date, therefore, it will be included as timedelta to capture the right date and time of arrival.

In [24]:
data["WHEELS_ON"] = data["WHEELS_OFF"] + data["AIR_TIME"].map_partitions(min_to_timedelta) + timezone

In [25]:
data["ARR_TIME"] = data["WHEELS_ON"] + data["TAXI_IN"].map_partitions(min_to_timedelta)

In [26]:
data["CRS_ARR_TIME"] = data["ARR_TIME"] - data["ARR_DELAY"].map_partitions(min_to_timedelta)

We will also convert distance column from mile to kilometer.

In [27]:
data["DISTANCE"] = data["DISTANCE"].map_partitions(lambda x: x*1.60934)
data = data.rename(columns = {"DISTANCE": "DISTANCE(km)"})

In [28]:
data.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE(km),CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2019-04-01 10:20:00,NK,BWI,MCO,2019-04-01 10:37:00,17.0,11.0,2019-04-01 10:48:00,2019-04-01 12:49:00,16.0,2019-04-01 12:45:00,2019-04-01 13:05:00,20.0,0.0,NaN,0.0,145.0,148.0,121.0,1266.55058,0.0,0.0,3.0,0.0,17.0
1,2019-04-01 07:55:00,NK,DTW,BWI,2019-04-01 08:30:00,35.0,12.0,2019-04-01 08:42:00,2019-04-01 09:42:00,6.0,2019-04-01 09:28:00,2019-04-01 09:48:00,20.0,0.0,NaN,0.0,93.0,78.0,60.0,658.22006,20.0,0.0,0.0,0.0,0.0
2,2019-04-01 19:31:00,NK,BWI,DTW,2019-04-01 19:46:00,15.0,8.0,2019-04-01 19:54:00,2019-04-01 21:05:00,29.0,2019-04-01 21:08:00,2019-04-01 21:34:00,26.0,0.0,NaN,0.0,97.0,108.0,71.0,658.22006,10.0,0.0,11.0,0.0,5.0
3,2019-04-01 16:23:00,NK,MCO,BWI,2019-04-01 16:51:00,28.0,11.0,2019-04-01 17:02:00,2019-04-01 18:41:00,5.0,2019-04-01 18:41:00,2019-04-01 18:46:00,5.0,0.0,NaN,0.0,138.0,115.0,99.0,1266.55058,0.0,0.0,0.0,0.0,0.0
4,2019-04-01 23:20:00,NK,LAS,DTW,2019-04-01 23:48:00,28.0,23.0,2019-04-02 00:11:00,2019-04-02 06:32:00,11.0,2019-04-02 06:22:00,2019-04-02 06:43:00,21.0,0.0,NaN,0.0,242.0,235.0,201.0,2814.73566,21.0,0.0,0.0,0.0,0.0


## Remove outliers

For the columns have minute as unit, we will check for outliers and remove them to make the analysis more accurate. These columns are:
- DEP_DELAY
- TAXI_OUT
- TAXI_IN
- ARR_DELAY
- CRS_ELAPSED_TIME
- ACTUAL_ELAPSED_TIME

And as mentioned before, there are still missing values in these columns which belongs to canceled flight, therefore, we will take non-canceled entries into consideration and assign the result to a new dataset variable named non_canceled_data. Canceled entries will be assigned to canceled_data.

To remove the outliers we are using Interquartile Range method. 
$$ IQR = Q_{3}- Q_{1}$$
Where $Q_{1}$ is the first quartile and $Q_{3}$ is the third quartile. From $IQR$ we will determine the low threshold and high thresh hold for each column and remove outliers that out side $[low\:thresholl, high\:threshold]$

In [29]:
outlier_check_col = ['DEP_DELAY', 'TAXI_OUT', 'TAXI_IN', 'ARR_DELAY', 'CRS_ELAPSED_TIME','ACTUAL_ELAPSED_TIME']
f_quartile = [data[(data["CANCELLED"]==0)&(data["DIVERTED"]==0)][i].quantile(0.25).compute() for i in outlier_check_col]
th_quartile = [data[(data["CANCELLED"]==0)&(data["DIVERTED"]==0)][i].quantile(0.75).compute() for i in outlier_check_col]
IQR = [i - j for i, j in zip(th_quartile, f_quartile)]
low_thres = [i - 1.5*j for i, j in zip(f_quartile, IQR)]
high_thres = [i + 1.5*j for i, j in zip(th_quartile, IQR)]
low_high_thres = pd.DataFrame(np.array([low_thres, high_thres]), columns=outlier_check_col)
low_high_thres

,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME
0,-34.0,-3.0,-3.5,-55.5,-36.0,-40.5
1,46.0,37.0,16.5,60.5,308.0,299.5


We now have the dataframe that contains the low and the high threshold for each columns.

In [30]:
non_canceled_data = data[(data["CANCELLED"]==0)&(data["DIVERTED"]==0)].copy()
canceled_data = data[(data["CANCELLED"]==1)|(data["DIVERTED"]==1)].copy()
for i in low_high_thres.columns:
    non_canceled_data = non_canceled_data[((non_canceled_data[i]>low_high_thres.loc[0,i])
                                          &(non_canceled_data[i]<low_high_thres.loc[1,i]))]                                        

In [31]:
print(f'There are {len(non_canceled_data)} entries in non_canceled_data')
print(f'There are {len(canceled_data)} entries in canceled_data')

There are 8555130 entries in non_canceled_data
There are 433201 entries in canceled_data


# Analyzing

In [32]:
df1 = non_canceled_data.set_index("FL_DATE")
df2 = canceled_data.set_index("FL_DATE")

## Busy time
### On basic of hour

In [33]:
busy_hour_no_cancel = df1.groupby(df1.index.hour).count()["OP_CARRIER"].compute()
busy_hour_cancel = df2.groupby(df2.index.hour).count()["OP_CARRIER"].compute()
total_busy_hour = busy_hour_no_cancel + busy_hour_cancel

In [34]:
total_busy_hour.to_frame().iplot(kind='bar', 
                                 title='Most busy hours for flights',
                                 xTitle='Hour',
                                 yTitle='Number of flights')

### On basic of day

In [35]:
busy_date_no_cancel = df1.groupby(df1.index.day).count()["OP_CARRIER"].compute()
busy_date_cancel = df2.groupby(df2.index.day).count()["OP_CARRIER"].compute()
total_busy_date = busy_date_no_cancel + busy_date_cancel

In [36]:
total_busy_date.iplot(kind='bar',
                      title='Air traffic by date in USA',
                      xTitle='Date',
                      yTitle='Number of flights')

### On basic of month

In [37]:
busy_month_no_cancel = df1.groupby(df1.index.month).count()["OP_CARRIER"].compute()
busy_month_cancel = df2.groupby(df2.index.month).count()["OP_CARRIER"].compute()
total_busy_month = busy_month_no_cancel + busy_month_cancel

In [38]:
total_busy_month.iplot(kind='bar',
                       title='Air traffic by months in USA',
                       xTitle='Months',
                       yTitle='Number of flights')

## Carrier

In [39]:
non_canceled_data['OP_CARRIER'].nunique().compute()

17

In [40]:
a = df1.groupby("OP_CARRIER").count()["ORIGIN"].compute()
b = df2.groupby("OP_CARRIER").count()["ORIGIN"].compute()
c = a+b 

In [41]:
c.to_frame().T.iplot(kind='bar', title = "Popularity of airline",
                    yTitle="Number of flight",
                    xTitle="Airline")

<mark>There are 17 different registered airlies in USA. Among them WN is the most popular airlines with more than 2 millions flights in 2019 and 2020 till September. HA is the least popular.

## Origin and Destination

In [42]:
a = non_canceled_data['ORIGIN'].nunique().compute()
b = non_canceled_data['DEST'].nunique().compute()

In [43]:
c = df1.groupby("ORIGIN").count()["OP_CARRIER"].compute()
d = df2.groupby("ORIGIN").count()["OP_CARRIER"].compute()
origin = c.add(d, fill_value=0)
e = df1.groupby("DEST").count()["OP_CARRIER"].compute()
f = df2.groupby("DEST").count()["OP_CARRIER"].compute()
des = e.add(f, fill_value=0)

### Ten most busiest airport

In [44]:
def two_level_categorize(part,thres1, thres2=0, level=2, *args):
    if level==2:
        for col in args:
            part[col] = part[col].apply(lambda x: 0 if x<=thres1 else 1)
        return part
    if level==3:
        for col in args:
            part[col] = part[col].apply(lambda x: 0 if x<=thres1 else (1 if x<=thres2 else 2))
        return part

In [45]:
origin.sort_values().tail(10).iplot(kind='bar',
                                title='Top 10 busiest airports by origin',
                                xTitle='Airports',
                                yTitle='Number of flights')

In [46]:
des.sort_values().tail(10).iplot(kind='bar',
                            title='Top 10 busiest airports by destination',
                            xTitle='Airports',
                            yTitle='Number of flights')

In [47]:
a = df1.map_partitions(two_level_categorize, 15,0,2, "ARR_DELAY").groupby("ORIGIN").sum()["ARR_DELAY"].compute()
b = df1.map_partitions(two_level_categorize, 15,0,2, "ARR_DELAY").groupby("DEST").sum()["ARR_DELAY"].compute()
origin1 = origin.sort_values().tail(10).index

In [48]:
z = (a[origin1]*100/origin[origin1]).to_frame(name="percentage").join(origin.to_frame(name="number of flight"))

In [49]:
z.iplot(kind='bar',secondary_y="number of flight",
       title="Top ten busiest airports and percentage of delay",
        yTitle='%',
        secondary_y_title="number of flight")

### Ten least busiest airport

In [50]:
des.sort_values().head(10).iplot(kind='bar',
                                title='Top ten least busiest airports by destination',
                                xTitle='Airports By Destinations',
                                yTitle='Number of flights')

In [51]:
origin.sort_values().head(10).iplot(kind='bar',
                                title='Top ten least busiest airports by origin',
                                xTitle='Airports By Destinations',
                                yTitle='Number of flights')

In [52]:
origin2 = ["IPT","CDB","PGV","HYA","BFM","DIK","GST","OWB","ADK","OGD"]
z = (a[origin2]*100/origin[origin2]).to_frame(name="percentage").join(origin.to_frame(name="number of flight"))

In [53]:
z.iplot(kind='bar', secondary_y="number of flight",
        title="Top ten least busiest airports and percentage of delay",
        yTitle='%',
        secondary_y_title="number of flight")

<mark>There were 367 different origins and destinations of flights. The most busiest airport was Hartsfield–Jackson Atlanta International Airport (ATL) with more than 5 hundred thousands flights in that period. While the least busy was Pago Pago International Airport (PPG) with only 3 flights in the same period.

## Departure delay time

In [54]:
a = non_canceled_data['DEP_DELAY'].describe().compute()
b = a.loc[["min", "mean", "max"]]

In [55]:
b.iplot(kind='bar', title = "Statistic for departure delay time", yTitle="minute")

## Which airline is the most delay

In [56]:
most_delay = non_canceled_data.groupby('OP_CARRIER').DEP_DELAY.mean().compute()

In [57]:
most_delay.to_frame().T.iplot(kind='bar',
                                title='Average delays for airlines in USA',
                                xTitle='Airlines',
                                yTitle='Time in minutes')

## Taxi-out

In [58]:
taxi_out = non_canceled_data.groupby('OP_CARRIER').TAXI_OUT.mean().compute()

In [59]:
taxi_out.iplot(kind='bar',
                title='Average TAXI_OUT time for airlines in USA',
                xTitle='Airlines',
                yTitle='Time in minutes')

In [60]:
taxi_out1 = non_canceled_data.groupby('ORIGIN').TAXI_OUT.mean().compute()

In [61]:
taxi_out1.sort_values().tail(10).iplot(kind='bar',
                title='Top 10 longer average TAXI_OUT time for airports in USA',
                xTitle='Airlines',
                yTitle='Time in minutes')

In [62]:
taxi_out1.sort_values().head(10).iplot(kind='bar',
                title='Top 10 shorter average TAXI_OUT time for different airports in USA',
                xTitle='Airlines',
                yTitle='Time in minutes')

## Canceled flight
### Number of canceled flight

In [63]:
total_flight = df1.resample("MS").count()["CANCELLED"].compute() + df2.resample("MS").count()["CANCELLED"].compute()
num_canceled = df2.resample("MS").sum()["CANCELLED"].compute()
num_diverted = df2.resample("MS").sum()["DIVERTED"].compute()

In [64]:
a = total_flight.to_frame(name='Total Flight')
b = a.groupby(a.index.year).mean()
c = total_flight.to_frame(name="Total Flight").join(num_canceled.to_frame(name="Num of Cancelled"))
d = num_canceled*100/total_flight

In [65]:
fig = px.bar(b, x=b.index,y="Total Flight")
fig.update_layout(title = "Average number of flight for 2019 and 2020", 
                  yaxis_title = "Number of flight",
                  xaxis =dict(title="Year", 
                  tickmode='array',
                  tickvals=[2019, 2020],
                  ticktext = b.index))                 
fig.show()

In [66]:
c.iplot(kind='bar',title="Total flight and cancelled flight for all regions of the U.S", yTitle='Number of flight')

The average number of flight of the U.S in 2019 is approximately 486,000, while this number up to September 2020 is only 349,000. Due to pandemic, and the President's on traveling restriction, the number of flight from April to September dropped significantly in comparison to same period last year. 
- An order on Jan. 31 restricting travel to the U.S. from China.  
- An additional order on March 11 restricting travel to the U.S. from certain European countries.

Furthermore, the number of canceled flight also spiked up for March and April 2020.

In [67]:
d.iplot(kind='bar', title='Percentage of cancelled flight', yTitle='%')

After the peak in April, the percentage of canceled flight decreased and eventually went down under 1% since the number of flight decreased and scheduled flight in this period is either important or less likely to be canceled due to subjective factors.
### Proportion of flight cancelation reasons

In [68]:
a = df2.query("CANCELLED==1").groupby("CANCELLATION_CODE").count()["OP_CARRIER"].compute()
b = a*100/a.values.sum()

In [69]:
fig = px.pie(b, values=b.values, color=b.index, names=["Carrier", "Weather", "NAS","Security"],
            title="Proportion of flight cancelation reasons")
fig.show()                

The most common reason for a flight to be canceled is security. From 2019 to 2020, over 57% of canceled flight is due to security. The second factor that affects the probability of flight cancellation is the weather, it is over 21%. Next one is due to Carrier with 13% and National Airspace System failure has the lowest percentage of 7%.

## Diverted flight

In [70]:
a = num_diverted*100/total_flight

In [71]:
a.iplot(kind='bar', title='Percentage of diverted flight', yTitle='%', hline=dict(y=a.mean(), dash='dash'))

The chance of a flight to be diverted is quite low (average of 0.2%)
## Elapsed Time

In [72]:
a = non_canceled_data.sample(frac=0.0005)[["CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME"]].compute()

In [73]:
a.iplot(x="CRS_ELAPSED_TIME", y="ACTUAL_ELAPSED_TIME", mode="markers",bestfit=True, bestfit_colors=["Blue"],
       title ="Relationship between scheduled elapsed time and actual elapsed time",
       xTitle = "CRS elapsed time",
       yTitle = "Actual elapsed time")

## Delay factors
### What is the most popular reason for delay

In [74]:
a = df1[df1["ARR_DELAY"]>15]
b = np.count_nonzero(a[["CARRIER_DELAY","WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].compute(), axis=0)
c = pd.DataFrame(b, index=["CARRIER_DELAY","WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"])

In [75]:
fig = px.bar(c, x=c.index,y=0,color=c.index)
fig.update_layout(title = "number of delayed flight for each factor", 
                  yaxis_title = "Number of flight",
                  xaxis =dict(title="Factor", 
                  tickmode='array',
                  tickvals=c.index,
                  ticktext = c.index))
fig.show()

Three factors: carrier delay, NAS delay, and late aircraft delay are the most popular reasons for flight to be delayed.
- Carrier delay can be due to: aircraft cleaning, aircraft damage, awaiting the arrival of connecting passengers or crew, baggage, cargo loading...
- Delay that is within the control of the National Airspace System (NAS) may include: non-extreme weather conditions, airport operations, heavy traffic volume, air traffic control...
- Arrival delay at an airport due to the late arrival of the same aircraft at a previous airport. The ripple effect of an earlier delay at downstream airports is referred to as delay propagation.
### The Longest delay period for each reason

In [76]:
b = a[["ARR_DELAY","CARRIER_DELAY","WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].describe().compute()
c = b.loc[["min","mean","max"],:]

In [77]:
c.T.iplot(kind='bar', title = "Duration statistic for each reason of delay",
         xTitle = "Reason",
         yTitle = "Minute")

The maximum delay time is 60 minutes and the minimum is, of course, 16 minutes since when the delay time less than 15 minutes, it is not considered as delayed.
## Time of the day and delay

In [78]:
a = non_canceled_data[non_canceled_data["FL_DATE"].dt.year == 2019]
b = a.groupby(a["FL_DATE"].dt.hour).count().compute()["FL_DATE"]
c = a.map_partitions(two_level_categorize, 15, 0, 2, "ARR_DELAY").groupby(a["FL_DATE"].dt.hour).sum()["ARR_DELAY"].compute()
d = c*100/b

In [79]:
d.iplot(kind='bar', title="Probability of being delay based on time of the day",
       xTitle="Time", yTitle = "Percentage")

We can see here, the highest probability for flight to be delay is at 21 and the lowest is at 5. Let's examine further the among the reason.

In [80]:
e = a.map_partitions(two_level_categorize, 0, 0, 2, "CARRIER_DELAY","WEATHER_DELAY", "NAS_DELAY",
                     "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY").groupby(a["FL_DATE"].dt.hour).sum().compute()

f = 100*e[["CARRIER_DELAY","WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].div(b, axis=0)

In [81]:
fig = px.bar(f, title="Probability of flight delay on basic of hour and reason")
fig.update_layout(yaxis_title = "%",
                  xaxis_title = "Hour")
fig.show()

In [82]:
f.T.iplot(kind='bar', title="Delay period on basic of hour and reason",
         xTitle="Reason",
         yTitle="Probability")

Based on the figure, we can see the probability for any delay reason to happen on 5 a.m is the lowest while the chance is triple at 21 and it is also the highest probability of flight to be delayed.
## Distance and delay
A assumption is made that there is relationship between distance of the flight and its delay. To test if this hypothesis is true or not, we will check the correlation between these variable. Since the delay of flight is decided by minutes of delay, we will convert this column into dummy variable. The same for distance, we will categorize this column into 2 groups: 0 indicate short distance (<1,500 km) and medium distance (<4,800 km).

**Null hypothesis (H0):** There is no relationship between distance and delay.

**Alternative hypothesis (H1):** There is relationship between distance and delay.

In [83]:
a = non_canceled_data[["ARR_DELAY", "DISTANCE(km)"]].map_partitions(two_level_categorize, 15, 0, 2, "ARR_DELAY") \
                                                    .map_partitions(two_level_categorize, 1500, 0, 2, "DISTANCE(km)")

b = a.groupby("DISTANCE(km)").sum().compute()
c = a.groupby("DISTANCE(km)").count().compute()
# Create Contingency Table
d = b.rename({'ARR_DELAY': "Delayed"}, axis=1).join(c.rename({'ARR_DELAY': "Not Delayed"}))
d.rename({0:"short",1:"medium"}, inplace=True)

In [103]:
print("Contigency table")
print(d)

Contigency table
              Delayed  ARR_DELAY
DISTANCE(km)                    
short          390324    6177068
medium         171747    2378062


In [104]:
from scipy.stats import chi2_contingency

stat, p, dof, expected = chi2_contingency(d.values)

In [117]:
e = b.rename({'ARR_DELAY': "Delayed"}, axis=1).join(c.rename({'ARR_DELAY': "Not Delayed"}))
expected = pd.DataFrame(expected).rename({0:"Delayed",1:"Not Delayed"}, axis=1).rename({0:"Short",1:"Medium"}, axis=0)
print("Expected value table\n")
print(expected)

Expected value table

            Delayed  Not Delayed
Short   404876.5173   6.1625e+06
Medium  157194.4827   2.3926e+06


The expected value is the value when two variables are independent. When delay and distance are independent, we have:

$P(delay&distance)= P(delay)*P(distance)$

In [119]:
print("Chi-square value:", stat)
print("Degree of freedom:", dof)

Chi-square value: 1993.024396262895
Degree of freedom: 1


In [105]:
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
    print('Dependent (reject H0)') 
else: 
    print('Independent (H0 holds true)') 

p value is 0.0
Dependent (reject H0)


Since the p-value is smaller than alpha 0.05, we reject the null hypothesis, which means that there is relationship between the distance and delay.